## Change Tracking

This is a cool trick I learned from [@lee215](https://leetcode.com/problems/maximum-sum-obtained-of-any-permutation/discuss/854206/JavaC++Python-Sweep-Line) on LeetCode for efficiently handling some types of overlapping interval problems. 

### Problem
Suppose we're given an array `ranges` where each element represents the start and end value of a range of sequential integers; `[1, 5]` stands for `[1,2,3,4,5]`. For each integer in each range, return a count of the number of ranges it is in.

**Example 1:**
```
Input: ranges = [[1,3], [2,4]]
Output: {1:1, 2:2, 3:2, 4:1}
Explanation: [1,3] means [1,2,3], and [2,4] means [2,3,4]. 1 is in one range, 2 is in two ranges, 
3 is in two ranges, and 4 is in one range. 
```
**Example 2:**
```
Input: [[1,3], [1,4], [2,4], [2,5]]
Output: {1:2, 2:4, 3:4, 4:3, 5:1}
Explanation: Our ranges are [1,2,3], [1,2,3,4], [2,3,4], [2,3,4,5]. 1 is in one range, 2 is in four ranges, etc. 
```

Some constraints:
- `0 <= len(ranges) <= 10**5`
- For every `ranges[i]`:
    - `0 <= ranges[i][0] <= ranges[i][1] <= 10**5`
    - `len(ranges[i]) == 2`

### Brute force approach
Using a counter, we could iterate through every value in every range and increment its frequency. This approach is `O(n^2)`; our worst case is `0 <= n <= 10**5` ranges each covering `0 <= n <= 10**5` values.

In [1]:
from collections import Counter

def count_frequencies(ranges):
    frequencies = Counter()
    for start, end in ranges:
        for i in range(start, end+1):
            frequencies[i]+=1
    return frequencies
        
for val, count in count_frequencies([[1,3], [1,4], [2,4], [2,5]]).items():
    print(f"{val}: {count}")

1: 2
2: 4
3: 4
4: 3
5: 1


## Tracking changes technique

Consider that each time we see a range, all values between the start and the end have are now present in one range (plus any others). We can only get around the quadratic complexity by skipping these middle values. So instead, what if we keep track of the boundaries of each range, and then just track the number of times an interval begins or ends on a particular boundary? Then we can go linearly through the bounds and just look at the changes in the number of covering intervals at each index.

Let's keep a Counter `bounds_counts`; for each range given, `bounds_count[range[0]] += 1` and `bounds_count[range[0]+1] -= 1` (this may sound weird, but hang on). If our ranges are `[[1,3],[2,4]]`, then the correct result would be `{1:1, 2:2, 3:2, 4:1}`. Using this strategy, `bounds_counts` looks like the following:
```
[1,3] -> {1:1, 4:-1}
[2,4] -> {1:1, 2:1, 4: -1, 5:-1}
```
Then, if we initialize `frequency = 0` and `frequencies = {}`, and loop over `range(1,5)`, adding `bounds_counts[i]` to `frequency` each time a value is in `bounds_counts` and storing the result in `frequencies`, the following occurs:
```
val   frequency   frequencies
-        0            {}
1        1 (+1)       {1:1}
2        2 (+1)       {1:1, 2:2}
3        2 (+0)       {1:1, 2:2, 3:2}
4        1 (-1)       {1:1, 2:2, 3:2, 4:1}
5        0 (-1)       same as above; don't include values with frequency 0. 
```
so frequencies now correctly reflects the count of intervals covering each value, and calculates it in linear time. In code:

In [2]:
def count_frequencies(ranges):
    bounds_counts = Counter()
    for start, end in ranges:
        bounds_counts[start] += 1
        bounds_counts[end+1] -= 1
    bounds = bounds_counts.keys()
    first, last = min(bounds), max(bounds)
    
    count = 0
    frequencies = {}
    for val in range(first, last):
        count += bounds_counts[val] if val in bounds_counts else 0
        frequencies[val] = count
    return frequencies
        
for val, count in count_frequencies([[1,3], [1,4], [2,4], [2,5]]).items():
    print(f"{val}: {count}")

1: 2
2: 4
3: 4
4: 3
5: 1


## [1589. Maximum Sum Obtained of Any Permutation](https://leetcode.com/problems/maximum-sum-obtained-of-any-permutation/)

There are two tricks for this problem:
1. Whatever "permutation" we pick should have the largest numbers at the most frequently covered index. 
2. We should get the index frequencies using the interval sweep technique. 

In [3]:
%run ../leetcode/1589-maximum-sum-obtained-of-any-permutation/solution.py

## [1109. Corporate Flight Bookings](https://leetcode.com/problems/corporate-flight-bookings/)

In [4]:
%run ../leetcode/1109-corporate-flight-bookings/solution.py

## [1094. Car Pooling](https://leetcode.com/problems/car-pooling/)

In [5]:
%run ../leetcode/1094-car-pooling/solution.py